In [5]:
from utils import *

In [ ]:
tickers_path = 'data/tickers/sp500.json'
dst_path = 'data/stocks/'

In [ ]:
success, fail = download_data(tickers_path, dst_path)
print(fail)

In [ ]:
src_path = 'data/stocks/'
dst_path = 'data/'

In [ ]:
merge_csv(src_path, dst_path)

In [6]:
df = pd.read_csv('data/dataset.csv')
df = df[-6000:-80]
df = df.dropna(axis=1)
df = df.set_index('Date')

In [9]:

data_size = 5000
input_tickers = df.columns
output_tickers = df.columns

In [26]:
scaler, x, y, dic = dataa(df, data_size, input_tickers, output_tickers, step_size=1, input_size=60, output_size=20, feature_range=(0,1))
x, y = x[:,:,2:5], y[:,:,2:5]

set()
356 356
356 356


In [27]:
x_train, y_train, x_val, y_val, x_test, y_test = train_val_test_split(x,y)

In [28]:
x.shape, y.shape

((4920, 60, 3), (4920, 20, 3))

In [29]:
input_shape = (x.shape[1],x.shape[2])
output_shape = (y.shape[1], y.shape[2])
layers_info = [(50, 0.2, True), (40, 0.15, False)]

input_shape, output_shape

((60, 3), (20, 3))

In [30]:

def create_model2(input_shape, output_shape, layers_info, metrics):

    num_els = output_shape[0] * output_shape[1]

    model = Sequential()

    (units, dropout_rate, ret) = layers_info[0]

    model.add(LSTM(units=units,return_sequences=ret, input_shape=input_shape))
    model.add(Dropout(dropout_rate))

    
    for (units, dropout_rate, ret) in layers_info[1:]:

        model.add(LSTM(units=units, return_sequences=ret))
        model.add(Dropout(dropout_rate))

    model.add(Dense(units=num_els))
    model.add(Reshape(output_shape))
    
    model.compile(optimizer='adam',loss='mse',metrics=metrics)

    return model

In [66]:
from keras import backend
 
def var(y_true, y_pred, axis):
	return backend.mean(backend.sum((y_pred - y_true) - backend.mean(y_pred - y_true, axis=axis, keepdims=True), axis=axis, keepdims=True), axis=axis)

def mse(y_true, y_pred, axis=-1):
    return backend.mean(backend.square(y_pred - y_true), axis=axis)

def E(y_true, y_pred):
	return backend.sqrt(var(y_true, y_pred,axis=0) + mse(y_true, y_pred, axis=0))
    

In [67]:
model = create_model2(input_shape, output_shape, layers_info, metrics=[E, 'acc', mse])

In [68]:
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=3,
    batch_size=32,
    validation_data=(x_val, y_val)
)

Epoch 1/3
108/108 [==============================] - 20s 122ms/step - loss: 0.0433 - E: 0.1762 - acc: 0.7036 - mse: 0.0433 - val_loss: 0.0035 - val_E: 0.0547 - val_acc: 0.9380 - val_mse: 0.0035
Epoch 2/3
108/108 [==============================] - 11s 105ms/step - loss: 0.0086 - E: 0.0898 - acc: 0.8938 - mse: 0.0086 - val_loss: 0.0019 - val_E: 0.0412 - val_acc: 0.9537 - val_mse: 0.0019
Epoch 3/3
108/108 [==============================] - 12s 109ms/step - loss: 0.0061 - E: 0.0760 - acc: 0.9152 - mse: 0.0061 - val_loss: 0.0017 - val_E: 0.0384 - val_acc: 0.9558 - val_mse: 0.0017


In [ ]:
model_name = 'LSTM_test'
model.save('models/' + model_name + '/model')

In [ ]:
from matplotlib import pyplot as plt

def validation_plot(train_arr, val_arr, f=plt.show, type='loss'):

    # f= lambda: plt.savefig(path)

    plt.plot(train_arr)
    plt.plot(val_arr)
    plt.title('model train vs validation loss')
    plt.ylabel(type)
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper right')
    f()

In [ ]:
validation_plot(history.history['loss'],history.history['val_loss'])

In [3]:
df = pd.read_csv('data/dataset.csv')
df.set_index('Date')

tickers_set = []
tickers_set.append(get_tickers_in_range(df, (188,360)))
tickers_set.append(get_tickers_in_range(df, (360,651)))
tickers_set.append(get_tickers_in_range(df, (651,1000)))
tickers_set.append(get_tickers_in_range(df, (1000,2000)))
tickers_set.append(get_tickers_in_range(df, (2000,5000)))
tickers_set.append(get_tickers_in_range(df, (5000,'MAX')))


for i in range(6):
        
    with open('data/tickers/sets/set' + str(i+1) + '.json', 'w') as f:
        json.dump(tickers_set[i], f)

set()
